In [56]:
from aocd import get_data
from collections import defaultdict
from dataclasses import dataclass
from typing import Dict, List, Tuple
import re

@dataclass(frozen=True)
class Move:
    amount: int
    start: int
    end: int


def chunk(lst, n):
    return (lst[i:i + n] for i in range(0, len(lst), n))

def parseLine(line: str) -> List[str]:
    values = []
    position = 0
    while position < len(line):
        current = line[position:position+3]
        if current == '   ':
            values.append(None)
        else:
            values.append(current[1])
        position+=4
    return values

assert(parseLine("    [D]    ") == [None, 'D', None])
assert(parseLine("[N] [C]    ") == ['N', 'C', None])
assert(parseLine("[Z] [M] [P]") == ['Z', 'M', 'P'])

def parseMove(line: str) -> Move:
    m = map(int,re.findall(r"\d+", line))
    return Move(*m)
    
assert(parseMove("move 1 from 2 to 1") == Move(1,2,1))
assert(parseMove("move 3 from 1 to 3") == Move(3,1,3))

def parseInput(input: str) -> Tuple[Dict[int, List[chr]], List[Move]]:
    grid = defaultdict(list)
    moves = []

    initialGrid = []

    state = 0
    for line in input:
        if state == 0 and line[1] != '1':
            initialGrid.append(parseLine(line))
        elif state == 0 and line[1] == '1':
            state += 1
            continue
        elif state == 2:
            moves.append(parseMove(line))
        else:
            state += 1
            continue
    
    initialGrid.reverse()

    for row in initialGrid:
        for k,v in enumerate(row):
            if v:
                grid[k+1].append(v)


    return (dict(grid), moves)

def executeMove(grid: Dict[int, List[chr]], move: Move) -> None:
    [grid[move.end].append(grid[move.start].pop()) for _ in range(move.amount)]

def executeMoveAsBlock(grid: Dict[int, List[chr]], move: Move, debug=False) -> None:
    if debug:
        print(f"Start: {grid}")
        print(f"Move: {move}")
    items = grid[move.start][-move.amount:]
    [grid[move.start].pop() for _ in range(move.amount)]
    grid[move.end]+=items
    if debug:
        print(f"End: {grid}")

def getEnds(grid: Dict[int, List[chr]]) -> str:
    result = []
    for x in grid.values():
        result.append(x[-1])
    return ''.join(result)


In [57]:
input = """    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2"""

input = get_data(day=5, year=2022)

lines = input.splitlines()

grid, moves = parseInput(lines)
[executeMove(grid, m) for m in moves]
print(f"Part 1: {getEnds(grid)}")

grid, moves = parseInput(lines)
[executeMoveAsBlock(grid, m) for m in moves]
print(f"Part 2: {getEnds(grid)}")


Part 1: CVCWCRTVQ
Part 2: CNSCZWLVT
